In [10]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error


In [2]:
X = pd.read_csv('..\data\homes.csv', index_col='Id')
X_test = pd.read_csv('..\data\home_test.csv', index_col='Id')

In [3]:
# Remove rows with missing target, separate target from predictors
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice
X.drop(['SalePrice'], axis=1, inplace=True)

In [5]:
# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X,y, train_size = 0.8, test_size = 0.2, random_state = 0)

In [6]:
# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique()<10 and X_train_full[cname].dtype == 'object']

In [7]:
numeric_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

In [8]:
# Keep selected columns only
my_cols = low_cardinality_cols + numeric_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test[my_cols].copy()

In [9]:
# One-hot encode the data (to shorten the code, we use pandas)
X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test = pd.get_dummies(X_test)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)

***
## Step 1: Build model
In this step,  build and train first model with gradient boosting.

 - Begin by setting my_model_1 to an XGBoost model. Use the XGBRegressor class, and set the random seed to 0 (random_state=0). Leave all other parameters as default.
 - Then, fit the model to the training data in X_train and y_train.

In [11]:
# Define the model
my_model_1 = XGBRegressor(random_state = 0)
# Fit the model
my_model_1.fit(X_train, y_train)
# Get predictions
predictions_1 = my_model_1.predict(X_valid)

# mae 1
mae_1 = mean_absolute_error(y_valid, predictions_1)
mae_1 

17662.736729452055

***
## Step 2: Improve the model
 - change the default parameters (like n_estimators and learning_rate) to get better results.
 - fit the model to the training data in X_train and y_train.

In [12]:
# Define the model
my_model_2 = XGBRegressor(n_estimators=1000, learning_rate=0.05) 

# Fit the model
my_model_2.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)],
             verbose=False) 

# Get predictions
predictions_2 = my_model_2.predict(X_valid) 

# Calculate MAE
mae_2 = mean_absolute_error(y_valid, predictions_2)
mae_2

16802.965325342466